In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

class disp(object):
    template = '<div style="float: left;padding:10px;"> <b>[{0}]</b> {1}</div>'
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join( self.template.format(a, eval(a)._repr_html_()) 
                        for a in self.args)
def prt(*pVPK) :
    print(*pVPK, sep = '\n\n', end = '\n\n')
    
import pandas as pd
import numpy as np    

### [예제1] insert() 메서드 활용

In [3]:
# insert(loc, columns, value, allow_duplicates = False)
# all_duplicates=F : 이미 컬럼이 존재하면 ValueError 발생
# all_duplicates=T : 이미 존재하는 컬럼 있어도 중복 허용하여 insert

df = pd.DataFrame([['kim',30],['lee',20]], columns=['name','age'])
df

df.insert(0, 'score', 100) # 위치 0번, 컬럼명 score, 값 100 insert
df.insert(len(df.columns), 'id', range(1, len(df.index)+1))
# df.insert(0, 'score', 50) #error. 이미 score 컬럼 존재함
df.insert(0, 'score', 50, allow_duplicates=True)
df.insert(df.columns.get_loc('age'), 'new', -1) # df.columns.get_loc('age') : age컬럼 위치 획득 -> 그 자리에 'new' insert.
df

,name,age
0,kim,30
1,lee,20


,score,score,name,new,age,id
0,50,100,kim,-1,30,1
1,50,100,lee,-1,20,2


### [예제2] assign() 메서드 활용

In [ ]:
df = pd.DataFrame([['kim',30],['lee',20]], columns=['name','age'])
df

df = df.assign(grade = 3, sex = ['W','M'])
df = df.assign(**{'kor':[70,100], 'eng':[50, 80]})
df = df.assign(age = df['age'] + 1, name = df['name'].str.upper())
df = df.assign(tot = df['kor'] + df['eng'], avg = lambda x : x['tot'] / 2)
df

### [예제3] sr.update() 동작

np.nan(NaN) : 실수값. 연산속도가 빠르고 편리하므로 missing value에 대한 default 표기로 사용

NA : pandas의 다양한 type의 데이터 처리를 위한 missing value 표기

In [ ]:
# [1]

d = {'age':[10,20,np.nan],'sex':['W','M',np.nan]}
df = pd.DataFrame(d, index=['kim','lee','park'])
df

sr1 = pd.Series([15,25,35])
df['age'].update(sr1)
sr1; df

In [ ]:
# [2]

d = {'age':[10,20,np.nan],'sex':['W','M',np.nan]}
df = pd.DataFrame(d, index=['kim','lee','park'])
df

sr2 = pd.Series([33,44],index=['kim','park'])
df['age'].update(sr2)
sr2; df

In [ ]:
# [3]

d = {'age':[10,20,np.nan],'sex':['W','M',np.nan]}
df = pd.DataFrame(d, index=['kim','lee','park'])
df

sr3 = pd.Series([np.nan,'Male'],index=['kim','park'])  # 덮어쓰려는 데이터의 nan은 덮어쓰지 않음.
df['sex'].update(sr3)
    
sr3; df

### [예제 4] df.update() 동작

In [ ]:
# [1]

d = {'age':[10,20,np.nan],'sex':['W','M',np.nan]}
df = pd.DataFrame(d, index=['kim','lee','park'])
df

sr1 = pd.Series([np.nan,33,44], index=['kim','park','lee']) #?
df.update(sr1)

sr2 = pd.Series([np.nan,'Male','Female'], index=['kim','park','lee'], name='sex')
df.update(sr2) # overwrite = True

sr1; sr2; df

In [ ]:
# [2]

d = {'age':[10,20,np.nan],'sex':['M','W',np.nan]}
df = pd.DataFrame(d,index=['kim','lee','park'])
df

sr2 = pd.Series([np.nan,'Male','Female'], index=['kim','park','lee'], name='sex')
df.update(sr2, overwrite=False)

sr2; df

In [ ]:
# [3]

d = {'age':[10,20,np.nan],'sex':['M','W',np.nan]}
df = pd.DataFrame(d, index=['kim','lee','park'])
df

sr3 = pd.Series([np.nan,33,44], index=['kim','park','lee'], name='age') 
df.update(sr3, errors='raise') #error

sr3; df

### [예제5] other가 DF인 update()의 동작

In [ ]:
# [1]

d = {'age':[np.nan,20,30],'sex':['M','W',np.nan]}
df = pd.DataFrame(d, index=['kim','lee','park'])

d2 = {'age':[33,44,55], 'sex':[np.nan,'male','Female'],}
df2 = pd.DataFrame(d2,index=['kim','song','park'])

disp('df', 'df2')
df.update(df2)
disp('df')

In [ ]:
# [2]

d = {'age':[np.nan,20,30],'sex':['M','W',np.nan]}
df = pd.DataFrame(d, index=['kim','lee','park'])

d2 = {'age':[33,44,55], 'sex':[np.nan,'male','Female'],}
df2 = pd.DataFrame(d2,index=['kim','song','park'])

disp('df', 'df2')
df.update(df2, overwrite=False)
disp('df')

### [예제6] update()의 filter_func 옵션

In [ ]:
d = {'kor':[30,40,50],'eng':[80,70,60]}
df = pd.DataFrame(d,index=['kim','lee','park'])
d2 = {'kor':[35,45,55],'eng':[85,75,65]}
df2 = pd.DataFrame(d2, index=['kim','song','park'])

disp('df', 'df2')

def func(x):
    print(x,  x<70)
    return x<70 

df.update(df2, filter_func=func, overwrite=False) 
disp('df')

### [예제7] drop() 동작

In [ ]:
# [1] labels, axis 조합 (default : axis=0)

a = np.arange(25).reshape(5,5)
df = pd.DataFrame(a,columns=list('ABCDE'),index=list('ijklm'))

df1 = df.drop(labels='i')
df1 = df1.drop(['j','k']) 
df1 = df1.drop('A', axis=1) 
df1 = df1.drop(labels=['B','C'], axis=1) 
disp('df', 'df1')

In [ ]:
# [2] index, columns 조합

a = np.arange(25).reshape(5,5)
df = pd.DataFrame(a,columns=list('ABCDE'),index=list('ijklm'))

df2 = df.drop(index ='i') 
df2 = df2.drop(columns ='A') 
df2 = df2.drop(index=['j','k'], columns=['B','C'])   
disp('df', 'df2')

In [ ]:
# [3] 

a = np.arange(25).reshape(5,5)
df = pd.DataFrame(a,columns=list('ABCDE'),index=list('ijklm'))

df3 = df.drop(index=['i','j','k'], columns=['A','B','C'], inplace=True)
print(df3)    
df

In [ ]:
# [4]

a = np.arange(25).reshape(5,5)
df = pd.DataFrame(a,columns=list('ABCDE'),index=list('ijklm'))

df4 = df.drop(labels = 'x') #error
df5 = df.drop(index=['i','j','k','x'],columns=['A','B','C','Y'],errors='ignore')
disp('df', 'df5')

### [예제8] 중복 행 삭제

In [ ]:
# [1]

d = {'class':list('ABBA'),'name':['kim','lee','lee','kim'],'id':[100,200,300,100]}
df = pd.DataFrame(d)

r1 = df.drop_duplicates()
r2 = df.drop_duplicates(keep='last')
r3 = df.drop_duplicates(keep=False)
disp('df', 'r1', 'r2', 'r3')

In [ ]:
# [2]

d = {'class':list('ABBA'),'name':['kim','lee','lee','kim'],'id':[100,200,300,100]}
df = pd.DataFrame(d)

r4 = df.drop_duplicates(['class','name'])
r5 = df.drop_duplicates('name')
r6 = df['name'].drop_duplicates()
disp('df', 'r4', 'r5'); r6

### [예제9] select_dtypes() 동작

In [ ]:
# [1]

d = {'name':['kim','lee','park'],'age':[10,20,30],'height':[155.5, 165.5, 170.5],'weight':[55,65,75]}
df = pd.DataFrame(d)

r1 = df.select_dtypes(include = object)
r2 = df.select_dtypes(include = np.number)
print(df.dtypes)
disp('df', 'r1','r2')

In [ ]:
# [2]

d = {'name':['kim','lee','park'],'age':[10,20,30],'height':[155.5, 165.5, 170.5],'weight':[55,65,75]}
df = pd.DataFrame(d)

df['age'] = df['age'].astype(np.int32)
r3 = df.select_dtypes(include=[int, float])
print(df.dtypes)
disp('df', 'r3')

In [ ]:
# [3]

d = {'name':['kim','lee','park'],'age':[10,20,30],'height':[155.5, 165.5, 170.5],'weight':[55,65,75]}
df = pd.DataFrame(d)
r4 = df.select_dtypes(include= np.number, exclude=int)
disp('df', 'r4')

### [예제10] pop() 동작

* pop과 drop의 차이?

* drop은 삭제. pop은 뽑아내기. 내가 뽑아낸 컬럼만 보여줌

In [4]:
# [1]

sr1 = pd.Series([100,90,80], index=['kim','lee','park'], name='kor')
prt(sr1)
v = sr1.pop('kim')
prt(sr1, v)

kim     100
lee      90
park     80
Name: kor, dtype: int64

lee     90
park    80
Name: kor, dtype: int64

100



In [5]:
# [2]

df = pd.DataFrame({'kor':[100,90,80], 'eng':[40,50,60]}, index=['kim','lee','park'])
df
sr2 = df.pop('kor')
df; sr2

,kor,eng
kim,100,40
lee,90,50
park,80,60


,eng
kim,40
lee,50
park,60


kim     100
lee      90
park     80
Name: kor, dtype: int64

In [6]:
# [3]

df = pd.DataFrame({'kor':[100,90,80], 'eng':[40,50,60]}, index=['kim','lee','park'])
df
df.insert(1, 'kor2', df.pop('kor')-5 )
df

,kor,eng
kim,100,40
lee,90,50
park,80,60


,eng,kor2
kim,40,95
lee,50,85
park,60,75
